In [1]:
import os
import cv2
import numpy as np

def overlay_images(folder1, folder2, output_folder):
    # 確保輸出資料夾存在
    os.makedirs(output_folder, exist_ok=True)

    # 取得資料夾1的圖片名稱
    images_folder1 = set(os.listdir(folder1))
    images_folder2 = set(os.listdir(folder2))

    # 找出兩個資料夾內相同名字的圖片
    common_images = images_folder1.intersection(images_folder2)

    for image_name in common_images:
        # 讀取圖片
        img1_path = os.path.join(folder1, image_name)
        img2_path = os.path.join(folder2, image_name)

        img1 = cv2.imread(img1_path, cv2.IMREAD_COLOR)
        img2 = cv2.imread(img2_path, cv2.IMREAD_COLOR)

        # 檢查圖片是否成功讀取且大小相同
        if img1 is None or img2 is None:
            print(f"無法讀取圖片: {image_name}")
            continue
        if img1.shape != img2.shape:
            print(f"圖片大小不一致: {image_name}")
            continue

        # 將兩張圖片重疊 (透明度可調整)
        blended = cv2.addWeighted(img1, 1, img2, 1, 0)

        # 儲存結果圖片
        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, blended)

    print(f"處理完成，重疊圖片已儲存至: {output_folder}")

# 設定資料夾路徑
folder1 = "/home/yuchi/Canny_Detection/crackforest/Output_1"
folder2 = "/home/yuchi/Canny_Detection/crackforest/Output_3"
output_folder = "/home/yuchi/Canny_Detection/crackforest/Output"

# 執行程式
overlay_images(folder1, folder2, output_folder)


處理完成，重疊圖片已儲存至: /home/yuchi/Canny_Detection/crackforest/Output


In [2]:

import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def read_images_with_matching(folder, prefix, file_extension):
    """读取指定资料夹中指定格式的图片，并匹配文件名前缀"""
    image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(file_extension)]
    images = {}
    for file in image_files:
        # 获取不含后缀的文件名，并去掉可能的 '_label'
        image_name = os.path.splitext(os.path.basename(file))[0].replace('_label', '')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # 读取灰度图像
        if image is not None:
            images[image_name] = image
        else:
            print(f"Warning: Could not read {file}")
    return images

def calculate_metrics(true_images, predicted_images, threshold=128):
    """计算 Precision, Recall 和 F-Measure"""
    precision_list, recall_list, f1_list = [], [], []
    
    for name, true_image in true_images.items():
        if name in predicted_images:
            pred_image = predicted_images[name]
            
            # 确保两张图片尺寸一致
            if true_image.shape != pred_image.shape:
                print(f"Skipping {name}: Shape mismatch ({true_image.shape} vs {pred_image.shape})")
                continue
            
            # 二值化处理
            true_binary = (true_image > threshold).astype(np.uint8).flatten()
            pred_binary = (pred_image > threshold).astype(np.uint8).flatten()
            
            # 计算指标
            precision = precision_score(true_binary, pred_binary, zero_division=0)
            recall = recall_score(true_binary, pred_binary, zero_division=0)
            f1 = f1_score(true_binary, pred_binary, zero_division=0)
            
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
        else:
            print(f"Warning: Missing matching image for {name} in predicted folder")
    
    # 计算平均指标
    avg_precision = np.mean(precision_list) if precision_list else 0
    avg_recall = np.mean(recall_list) if recall_list else 0
    avg_f1 = np.mean(f1_list) if f1_list else 0
    
    return avg_precision, avg_recall, avg_f1

def main(jpg_folder, png_folder, threshold=128):
    """主函数"""
    # 读取两个文件夹中的图片
    print("Reading .jpg images...")
    true_images = read_images_with_matching(jpg_folder, '', '.jpg')
    
    print("Reading .PNG images...")
    predicted_images = read_images_with_matching(png_folder, '_label', '.PNG')
    
    # 计算 Precision, Recall 和 F-Measure
    print("Calculating metrics...")
    precision, recall, f1 = calculate_metrics(true_images, predicted_images, threshold)
    
    # 输出结果
    print("\nResults:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F-Measure: {f1:.4f}")

# 示例运行
if __name__ == "__main__":
    jpg_folder = "/home/yuchi/Canny_Detection/crackforest/Output"  # 替换为您的 .jpg 图片资料夹路径
    png_folder = "/home/yuchi/Canny_Detection/crackforest/Masks_copy"  # 替换为您的 .PNG 图片资料夹路径
    main(jpg_folder, png_folder, threshold=128)


Reading .jpg images...
Reading .PNG images...
Calculating metrics...

Results:
Precision: 0.6063
Recall: 0.3608
F-Measure: 0.4369
